In [2]:
import pandas as pd
import numpy as np

In [4]:
#now let's do some data balancing,tokenization and padding in this section

df=pd.read_csv("../data/new_dataset/combined_data.csv")
df["label"].value_counts()

label
3    1918
0    1755
2    1397
1    1118
Name: count, dtype: int64

In [6]:
def create_balanced_df(df):
    min_count=df["label"].value_counts().min()
    balanced_df = df.groupby('label').sample(n=min_count, random_state=42)
    return balanced_df
balanced_df=create_balanced_df(df)
balanced_df["label"].value_counts()
balanced_df.head()

,tweet,label
322,fatan mu ayi shi lafiya a gama lafiya allah ya...,0
408,may allah grant you whatever your beautiful he...,0
405,irin wannan wanka hakah,0
462,yau dai ana baka kuluwaa,0
1137,hakadai zaayi tayi mudai allah shine yake kare...,0


In [9]:
#this code block shuffles and resets the index of the balanced dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df.head()

,tweet,label
0,i am a proud igbo girl ọ dị nọbara m i am unas...,1
1,allah ya kaimu aljannah mu hada arewaaljannah ...,0
2,update first dance video for this year drops t...,2
3,oluwa a pano mo ase mo fẹ́ dé ọ̀dọ̀ ìbátan mi ...,3
4,i follow back ndi imo ndewo,1


In [10]:
#now let's create a train, validation and test split
#let first see the length of our dataframe
len(balanced_df)

4472

In [12]:
def create_split(df,train_ratio,test_ratio,val_ratio):
    train_end=int(len(df)*train_ratio)
    val_end=train_end+int(len(df)*val_ratio)
    train_df=df[:train_end]
    val_df=df[train_end:val_end]
    test_df=df[val_end:]
    return train_df,val_df,test_df
train_df,val_df,test_df=create_split(balanced_df,0.7,0.15,0.15)

In [13]:
train_df.to_csv("../data/new_dataset/train_data.csv",index=False)
val_df.to_csv("../data/new_dataset/val_data.csv",index=False)
test_df.to_csv("../data/new_dataset/test_data.csv",index=False)

In [19]:
import pandas as pd
import tiktoken
import torch

class Tokenizer:
    def __init__(self, file, tokenizer, max_len=None, pad_token_id=50256):
        # Load dataset
        self.file = pd.read_csv(file)
        self.tokenizer = tokenizer
        self.pad_token_id = pad_token_id
        
        # Encode all texts
        self.encodings = [self.tokenizer.encode(text) for text in self.file["tweet"]]
        
        # Determine max length
        if max_len is None:
            self.max_len = self.get_max_len()
        else:
            self.max_len = max_len
        
        # Truncate and pad sequences
        self.encodings = [
            enc[:self.max_len] + [self.pad_token_id] * (self.max_len - len(enc[:self.max_len]))
            for enc in self.encodings
        ]

    # PyTorch-style getter
    def __getitem__(self, idx):
        item = self.encodings[idx]
        label = self.file.iloc[idx]["label"]
        return torch.tensor(item), torch.tensor(label)

    # PyTorch-style length
    def __len__(self):
        return len(self.encodings)

    # Compute max sequence length from data
    def get_max_len(self):
        max_len = max(len(enc) for enc in self.encodings)
        return max_len


# Usage
tokenizer = tiktoken.get_encoding("gpt2")
train_tokenizer = Tokenizer("../data/new_dataset/train_data.csv", tokenizer=tokenizer)
train_max_len = train_tokenizer.get_max_len()
print("Max length in dataset:", train_max_len)


Max length in dataset: 277


In [24]:
train_dataset=Tokenizer("../data/new_dataset/train_data.csv",tokenizer=tokenizer,max_len=None)
val_dataset=Tokenizer("../data/new_dataset/val_data.csv",tokenizer=tokenizer,max_len=train_dataset.max_len)
test_dataset=Tokenizer("../data/new_dataset/test_data.csv",tokenizer=tokenizer,max_len=train_dataset.max_len)

In [25]:
#Now let's create dataloaders for our datasets
from torch.utils.data import DataLoader
torch.manual_seed(42)
train_loader=DataLoader(
    dataset=train_dataset,
    batch_size=8,
    num_workers=0,
    drop_last=True,
    shuffle=True
)
test_loader=DataLoader(
    dataset=test_dataset,
    batch_size=8,
    num_workers=0,
    drop_last=True,
    shuffle=True
)
val_loader=DataLoader(
    dataset=val_dataset,
    batch_size=8,
    num_workers=0,
    drop_last=True,
    shuffle=True
)

In [29]:
for input,label in train_loader:
    print(input.shape)
    print(label.shape)
    

torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
torch.Size([8, 277])
torch.Size([8])
t

In [30]:
print(len(train_loader))

391
